<h1 style="text-align: center; font-weight: bold;">Data Sciense Course - Spring 1403</h1>
<h1 style="text-align: center; font-weight: bold;">Project Phase1 - Introduction to Data Science </h1>
<h1 style="text-align: center;">Mohammadreza Mohammadhashemi : 810100206</h1>
<h1 style="text-align: center;">Soheil Hajian Manesh : 810100119</h1>
<h1 style="text-align: center;">Mahdi Ebrahimi Soltani : 810100241</h1>


# Problem Description
in this phase we are going to scrapping datas from fiverr.com website using BeautifulSoup and selenium packages and then extract some meaningful statistics from dataset using EDA and visualisation and preprocessing steps.

In [4]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import os

## Data Collection

In [1]:
import os
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

def initialize_driver(proxy=None):


    options = Options()
    options.add_argument(r"user-data-dir=C:\Users\Lenovo\AppData\Local\Google\Chrome\User Data")
    options.add_argument(r"profile-directory=Profile 1")
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36"
    options.add_argument(f"user-agent={user_agent}")
    driver = webdriver.Chrome(options=options)
    return driver


base_url = "https://www.fiverr.com"

In [2]:
def get_text_or_default(element, default="N/A"):
    return element.text.strip() if element else default

In [3]:
def extract_price_info(table):
    prices = table.find_all("div", class_="price-wrapper")
    return (
        get_text_or_default(prices[0]),
        get_text_or_default(prices[1]),
        get_text_or_default(prices[2]),
    )

In [4]:
def extract_delivery_info(table):
    row = table.find("tr", class_="delivery-time")
    tds = row.find_all("td")
    delivery_0 = tds[1].find_all("label")[0] if len(tds[1].find_all("label"))>0 else tds[1]
    delivery_1 = tds[2].find_all("label")[0] if len(tds[2].find_all("label"))>0 else tds[2]
    delivery_2 = tds[3].find_all("label")[0] if len(tds[3].find_all("label"))>0 else tds[3]
    return (
            get_text_or_default(delivery_0),
            get_text_or_default(delivery_1),
            get_text_or_default(delivery_2),
        )

In [5]:
def extract_revision_info(table):
    rows = table.find_all("tr")
    revisions = rows[9].find_all("td") if len(rows) > 9 else []
    return (
        get_text_or_default(revisions[1]),
        get_text_or_default(revisions[2]),
        get_text_or_default(revisions[3]),
    )

In [6]:
def extract_user_stats(soup):
    user_stats = soup.find("ul", class_="user-stats").find_all("li")
    return (
        get_text_or_default(user_stats[0].find("strong")),
        get_text_or_default(user_stats[1].find("strong")),
        get_text_or_default(user_stats[2].find("strong")),
        get_text_or_default(user_stats[3].find("strong")),
        get_text_or_default(user_stats[4].find("strong")),
    )

In [7]:
def extract_rating_info(soup):
    rating = get_text_or_default(soup.find("b", class_="rating-score"))
    rating_count = get_text_or_default(soup.find("span", class_="rating-count-number"))
    return rating, rating_count

In [8]:
def extract_order_in_queue(soup):
    order_in_queue = soup.find("div", class_="FVVUrQM")
    return get_text_or_default(order_in_queue)

In [9]:
import time
from selenium.common.exceptions import WebDriverException
def extract_records(driver, url,category, field , level, max_retries=3):

    for attempt in range(max_retries):

        try:

            driver.get(url)
    
            body = driver.find_element(By.CLASS_NAME, "layout-service")

            html_content = body.get_attribute("innerHTML")

            soup = BeautifulSoup(html_content, "html.parser")
        
            table = soup.find("div", class_="gig-page-packages-table")

            basic_price, standard_price, premium_price = extract_price_info(table)

            basic_delivery, standard_delivery, premium_delivery = extract_delivery_info(table)

            basic_revision, standard_revision, premium_revision = extract_revision_info(table)
            

            rating, rating_count = extract_rating_info(soup)

            country, member_since, avg_response_time, last_delivery, language = extract_user_stats(soup)

            order_in_queue = extract_order_in_queue(soup)
            

            return (
                category, field, level, 
                basic_price, standard_price, premium_price,
                basic_delivery, standard_delivery, premium_delivery,
                basic_revision, standard_revision, premium_revision,
                rating, rating_count, country, member_since,
                avg_response_time, last_delivery, language, order_in_queue

            )

        except WebDriverException as e:

            print(f"Attempt {attempt + 1} failed: {e}")

            time.sleep(2)  # Optional: wait before retrying

            driver.refresh()

        except Exception as e:

            print(f"An unexpected error occurred: {e}")

            break

    return None 

In [10]:
# def extract_records(driver, url):
#     driver.get(url)
#     body = driver.find_element(By.CLASS_NAME,"layout-service")
#     html_content = body.get_attribute("innerHTML")
#     soup = BeautifulSoup(html_content, "html.parser")
#     table = soup.find("div", class_="gig-page-packages-table")
#     prices = table.find_all("div", class_="price-wrapper")
#     basic_price = prices[0].text.strip()
#     standard_price = prices[1].text.strip()
#     premium_price = prices[2].text.strip()
#     deliveries = table.find_all("label", class_="jQREp61")
#     basic_delivery = deliveries[0].text.strip()
#     standard_delivery = deliveries[2].text.strip()
#     premium_delivery = deliveries[4].text.strip()
#     revisions = (table.find_all("tr"))[9].find_all("td")
#     basic_revision = revisions[1].text.strip()
#     standard_revision = revisions[2].text.strip()
#     premium_revision = revisions[3].text.strip()
#     rating = soup.find_all("b", class_="rating-score")[0].text.strip()
#     rating_count = soup.find_all("span", class_="rating-count-number")[0].text.strip()
#     user_stats = soup.find("ul",class_="user-stats").find_all("li")
#     country = user_stats[0].find("strong").text.strip()
#     member_since = user_stats[1].find("strong").text.strip()
#     avg_response_time = user_stats[2].find("strong").text.strip()
#     last_delivery = user_stats[3].find("strong").text.strip()
#     language = user_stats[4].find("strong").text.strip()
#     order_in_queue = soup.find("div", class_="FVVUrQM").text.strip()
#     print(f"basic price:{basic_price}")
#     print(f"Sta price:{standard_price}")
#     print(f"pre price:{premium_price}")
#     print(f"base deliv:{basic_delivery}")
#     print(f"stan deliv:{standard_delivery}")
#     print(f"prem deliv:{premium_delivery}")
#     print(f"base revis:{basic_revision}")
#     print(f"stan revis:{standard_revision}")
#     print(f"prem revis:{premium_revision}")
#     print(f"rating:{rating}")
#     print(f"rate_count:{rating_count}")
#     print(f"country:{country}")
#     print(f"mem_since:{member_since}")
#     print(f"avg_res_time:{avg_response_time}")
#     print(f"last del:{last_delivery}")
#     print(f"lang:{language}")
#     print(f"in queue:{order_in_queue}")

In [11]:
# category_fields_mapping = {
#     "Graphics & Design": {
#         "/categories/graphics-design/game-art",
#         "/categories/graphics-design/social-media-design",
#         "/categories/graphics-design/fashion-design",
#     },
#     "Music & Audio": {
#         "/categories/music-audio/mixing-mastering",
#         "/categories/music-audio/producers",
#         "/categories/music-audio/singers-vocalists",
#         "/categories/music-audio/songwriters",
#     },
#     "Programming & Tech": {
#         "/categories/programming-tech/website-development",
#         "/categories/programming-tech/software-development",
#         "/categories/programming-tech/mobile-app-services",
#         "/categories/programming-tech/game-development",
#     },
#     "Digital Marketing": {
#         "/categories/online-marketing/social-marketing",
#         "/categories/online-marketing/seo-services",
#         "/categories/online-marketing/online-video-marketing",
#     },
#     "Business": {
#         "/categories/business/financial-consulting-services",
#         "/categories/business/sales",
#         "/categories/business/business-plans",
#         "/categories/business/software-management",
#     },
#     "Writing & Translation": {
#         "/categories/writing-translation/articles-blogposts",
#         "/categories/writing-translation/quality-translation-services",
#         "/categories/writing-translation/book-editing",
#     },
#     "Data": {
#         "/categories/data/data-mining",
#         "/categories/data/databases",
#         "/categories/data/data-engineering",
#         "/categories/data/data-science",
#     },
#     "Photography": {
#         "/categories/photography/product-photographers",
#         "/categories/photography/event-photographers",
#         "/categories/photography/real-estate-photographers",
#     },
#     "Lifestyle": {
#         "/categories/lifestyle/online-tutoring",
#         "/categories/lifestyle/personal-stylists",
#         "/categories/lifestyle/fitness",
#         "/categories/lifestyle/modeling-acting",
#     },
#     "Video & Animation": {
#         "/categories/video-animation/video-editing",
#         "/categories/video-animation/visual-effects",
#         "/categories/video-animation/animated-explainer-videos",
#     },
# }

In [13]:
def process_html_files(base_path):
    records = []
    driver = initialize_driver()
    for category in os.listdir(base_path):
        category_path = os.path.join(base_path, category)
        if os.path.isdir(category_path):
            for field in os.listdir(category_path):
                field_path = os.path.join(category_path, field)
                if os.path.isdir(field_path):
                    for file_name in ["0.html", "1.html", "2.html", "3.html"]:
                        file_path = os.path.join(field_path, file_name)
                        if os.path.isfile(file_path):
                            with open(file_path, "r", encoding="utf-8") as file:
                                html_content = file.read()
                            soup = BeautifulSoup(html_content, "html.parser")
                            item_divs = soup.find_all("div", class_="gig-card-layout")
                            item_links = []
                            for div in item_divs:
                                link = div.find_all("a", href=True)[2]
                                if link:
                                    item_links.append(
                                        "https://www.fiverr.com" + link["href"]
                                    )
                            seller_level = [
                                "top rated seller",
                                "level 1",
                                "level 2",
                                "new seller",
                            ][int(file_name[0])]
                            for link in item_links:
                                record = extract_records(
                                    driver, link, category, field, seller_level
                                )
                                if record:
                                    records.append(record)

    driver.quit()
    return records

columns = [
    "Category",
    "Field",
    "Seller Level",
    "Basic Price",
    "Standard Price",
    "Premium Price",
    "Basic Delivery",
    "Standard Delivery",
    "Premium Delivery",
    "Basic Revision",
    "Standard Revision",
    "Premium Revision",
    "Rating",
    "Rating Count",
    "Country",
    "Member Since",
    "Avg Response Time",
    "Last Delivery",
    "Language",
    "Order in Queue",
]

base_path = "htmls"
records = process_html_files(base_path)
fiverr_df = pd.DataFrame(records, columns=columns)

print(fiverr_df)

An unexpected error occurred: 'NoneType' object has no attribute 'find_all'
Attempt 1 failed: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=125.0.6422.77)
Stacktrace:
	GetHandleVerifier [0x00007FF6F3A41F22+60322]
	(No symbol) [0x00007FF6F39BCE99]
	(No symbol) [0x00007FF6F3877EBA]
	(No symbol) [0x00007FF6F384D5A5]
	(No symbol) [0x00007FF6F38F36B7]
	(No symbol) [0x00007FF6F390B331]
	(No symbol) [0x00007FF6F38EBFC3]
	(No symbol) [0x00007FF6F38B9617]
	(No symbol) [0x00007FF6F38BA211]
	GetHandleVerifier [0x00007FF6F3D5946D+3301613]
	GetHandleVerifier [0x00007FF6F3DA3693+3605267]
	GetHandleVerifier [0x00007FF6F3D99410+3563664]
	GetHandleVerifier [0x00007FF6F3AF42F6+790390]
	(No symbol) [0x00007FF6F39C74DF]
	(No symbol) [0x00007FF6F39C33D4]
	(No symbol) [0x00007FF6F39C3562]
	(No symbol) [0x00007FF6F39B2F6F]
	BaseThreadInitThunk [0x00007FFA3BB87344+20]
	RtlUserThreadStart [0x00007FFA3DA626B1+33]



NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=125.0.6422.77)
Stacktrace:
	GetHandleVerifier [0x00007FF6F3A41F22+60322]
	(No symbol) [0x00007FF6F39BCE99]
	(No symbol) [0x00007FF6F3877EBA]
	(No symbol) [0x00007FF6F384D5A5]
	(No symbol) [0x00007FF6F38F36B7]
	(No symbol) [0x00007FF6F390B331]
	(No symbol) [0x00007FF6F38EBFC3]
	(No symbol) [0x00007FF6F38B9617]
	(No symbol) [0x00007FF6F38BA211]
	GetHandleVerifier [0x00007FF6F3D5946D+3301613]
	GetHandleVerifier [0x00007FF6F3DA3693+3605267]
	GetHandleVerifier [0x00007FF6F3D99410+3563664]
	GetHandleVerifier [0x00007FF6F3AF42F6+790390]
	(No symbol) [0x00007FF6F39C74DF]
	(No symbol) [0x00007FF6F39C33D4]
	(No symbol) [0x00007FF6F39C3562]
	(No symbol) [0x00007FF6F39B2F6F]
	BaseThreadInitThunk [0x00007FFA3BB87344+20]
	RtlUserThreadStart [0x00007FFA3DA626B1+33]


In [96]:
# html_file_path = "1.html"

# with open(html_file_path, "r", encoding="utf-8") as file:
#     html_content = file.read()

# soup = BeautifulSoup(html_content, "html.parser")
# item_divs = soup.find_all("div", class_="gig-card-layout")
# item_links = []
# for div in item_divs:
#     link = div.find_all("a", href=True)[2]
#     if link:
#         item_links.append("https://www.fiverr.com" + link["href"])


# records = []
# driver = initialize_driver()
# for link in item_links:
#     record = extract_records(driver, link)
#     records.append(record)
# driver.quit()

In [97]:
# columns = [
#     "Rating",
#     "Rating Count",
#     "Order in Queue",
#     "Basic Plan Cost",
#     "Standard Plan Cost",
#     "Premium Plan Cost",
#     "Basic Plan Revisions",
#     "Standard Plan Revisions",
#     "Premium Plan Revisions",
#     "Basic Plan Delivery",
#     "Standard Plan Delivery",
#     "Premium Plan Delivery",
#     "Avg Response Time",
#     "Language",
#     "Country",
#     "Member Since",
#     "Last Delivery",
# ]

# fiverr_df = pd.DataFrame(records, columns=columns)

# driver.quit()

# print(fiverr_df)